# Deep Learning Model

# --------------------------------------------------------
## 1) Import packages

In [1]:
import pandas as pd
import numpy as np

import os
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras_tuner.tuners import RandomSearch


# pip install tensorflow
# pip install keras
# pip install keras-tuner

# --------------------------------------------------------
## 2) Load Dataset

In [2]:
df = pd.read_csv("../../Data/Student_performance_scaled.csv")

#drop GPA column
#df.drop(columns=['GPA'], inplace=True)

df.head()

,Age,Gender,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GradeClass
0,0.472919,0.978492,0.253711,1.780336,-0.890822,1.522371,-0.108744,-0.788476,-0.660132,2.019544,-0.431866,2.0
1,1.362944,-1.021981,-0.746087,0.997376,-1.717694,-0.656870,-0.999551,-0.788476,-0.660132,-0.495161,-0.431866,1.0
2,-1.307132,-1.021981,1.253509,-0.984045,1.353542,-0.656870,-0.108744,-0.788476,-0.660132,-0.495161,-0.431866,4.0
3,0.472919,0.978492,1.253509,0.045445,-0.063951,-0.656870,0.782063,1.268269,-0.660132,-0.495161,-0.431866,3.0
4,0.472919,0.978492,0.253711,-0.902311,0.290422,1.522371,0.782063,-0.788476,-0.660132,-0.495161,-0.431866,4.0


# --------------------------------------------------------
## 3) Feature Engineering

#### i) Encoding Categorical Variables: 

In [3]:
def encode_categorical_features(df):
    # all features are already scaled, so return unchanged.
    return df

#### ii) Ratio & Aggregate Features: 

adds new features in the for of ratios

`StudyAbsenceRatio` combines `StudyTimeWeekly` and `Absences`. ↑study:↓absent = ↑ratio

In [4]:
# Create ratio-based feature(s)
def add_ratio_features(df):
    df = df.copy()
    # Study Time to Absence ratio
    df['StudyAbsenceRatio'] = df['StudyTimeWeekly'] / (df['Absences'] + 1)  # +1 to avoid division by zero
    return df

#### iii) Interaction Features:

adds new features in terms of interaction

`SportsMusic` multiplies `Sports` and `Music` to give an understanding into the total extra carricular activities a student takes part in

`TotalSupport` adds `TotalSupport` and `Tutoring` to show total support given to a student

In [5]:
#optional
def add_interaction_features(df):
    df = df.copy()
    # Combining sports and music participation
    df['SportsMusic'] = df['Sports'] * df['Music']
    # Combined parental involvement
    df['TotalSupport'] = df['ParentalSupport'] + df['Tutoring']
    return df

#### iV) Apply all feature engineering:

In [6]:
def apply_feature_engineering(df):
    df = encode_categorical_features(df)
    df = add_ratio_features(df)
    df = add_interaction_features(df)
    return df

# --------------------------------------------------------
## 4) Prepare Data

`x` = features (independent variables the model learns from).

`y` = target (GradeClass, the label we want the model to predict).

In [7]:
# Apply feature engineering
df = apply_feature_engineering(df)

# Define features and target
X = df.drop(['GradeClass'], axis=1)
y = df['GradeClass']

# Split the data into train and test sets (ensure y_test is defined)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# --------------------------------------------------------
## 5) Build model and set up tuner

using the sequential keras model

Model: https://keras.io/api/models/sequential/

Explained: https://www.geeksforgeeks.org/keras-sequential-class/

In [8]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units1', min_value=32, max_value=256, step=32), activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(rate=hp.Float('dropout1', min_value=0.0, max_value=0.5, step=0.1)))
    
    model.add(Dense(units=hp.Int('units2', min_value=32, max_value=256, step=32), activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout2', min_value=0.0, max_value=0.5, step=0.1)))

    model.add(Dense(5, activation='softmax'))  # 5 classes for GradeClass

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


keras tuner:

https://keras.io/keras_tuner/api/tuners/random/

automatically searches for the best hyperparameters for the deep learning model instead of using a grid or manual methods.

It works by:
1) Randomly picks different combinations of settings.
2) Trains a model with each.
3) Picks the best based on a metric specified (`objective` = `'val_accuracy'`).

Settings it tries in the script:

`units1`, `units2`: Neurons in 1st and 2nd layers (32 to 256).

`dropout1`, `dropout2`: Dropout rates (0 to 0.5).

In [9]:
# Keras Tuner
tuner = RandomSearch(
    hypermodel=build_model,
    objective='val_accuracy',
    max_trials=10,
    seed=42,
    directory='../../Tuners/student_tuner',
    project_name='grade_classification'
)

tuner.search(X_train, y_train, epochs=20, validation_split=0.2)

Reloading Tuner from ../../Tuners/student_tuner\grade_classification\tuner0.json


# --------------------------------------------------------
## 6) Choose the best model and run predictions

Best model = the one that scored highest on validation accuracy during tuner search.

Fit the best model again on full training data (20 epochs).

Uuse it to predict the classes for X_test.

In [10]:
# Get best model
best_model = tuner.get_best_models(num_models=1)[0]

# Fit on full training data
best_model.fit(X_train, y_train, epochs=20, validation_split=0.2)

# Predictions
y_pred = np.argmax(best_model.predict(X_test), axis=1)


Epoch 1/20


c:\Program Files\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Program Files\Python311\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7846 - loss: 0.6528 - val_accuracy: 0.7154 - val_loss: 0.8478
Epoch 2/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7838 - loss: 0.6302 - val_accuracy: 0.7258 - val_loss: 0.8293
Epoch 3/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7971 - loss: 0.6334 - val_accuracy: 0.7285 - val_loss: 0.8304
Epoch 4/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8248 - loss: 0.5684 - val_accuracy: 0.7311 - val_loss: 0.8083
Epoch 5/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8062 - loss: 0.6094 - val_accuracy: 0.7389 - val_loss: 0.7907
Epoch 6/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8264 - loss: 0.5240 - val_accuracy: 0.7232 - val_loss: 0.8138
Epoch 7/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8175 - loss: 0.5136 - val_accuracy: 0.7363 - val_loss: 0.7996
Epoch 8/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8330 - loss: 0.5292 - val_accuracy: 0.7337 - val_loss: 0.8132
Epo

# --------------------------------------------------------
## 7) Run Evaluation Metrics

#### Accuracy

simple measure of correctness

`correct predictions` / `total predictions`

In [11]:
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.7056367432150313


#### Precision (weighted)

how many predictions were actually correct

weighted adjusts for class imbalance

In [12]:
print("Precision:", precision_score(y_test, y_pred, average='weighted'))

Precision: 0.7064131109286844


#### Recall (weighted)

how many labels were correctly predicted?

weighted adjusts for class imbalance

In [13]:
print("Recall:", recall_score(y_test, y_pred, average='weighted'))

Recall: 0.7056367432150313


#### F1 Score (weighted)

harmonic mean of precision and recall

weighted adjusts for class imbalance

In [14]:
print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))

F1 Score: 0.7033754138231846


#### Confusion Matrix

shows real vs predicted class counts

In [15]:
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Confusion Matrix:
 [[  4   7   0   5   5]
 [  9  21  19   3   2]
 [  1   7  49  20   1]
 [  0   1  21  47  14]
 [  1   3   2  20 217]]


#### Classification Report

breakdown of precision, recall, F1-Score per class

In [16]:
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

         0.0       0.27      0.19      0.22        21
         1.0       0.54      0.39      0.45        54
         2.0       0.54      0.63      0.58        78
         3.0       0.49      0.57      0.53        83
         4.0       0.91      0.89      0.90       243

    accuracy                           0.71       479
   macro avg       0.55      0.53      0.54       479
weighted avg       0.71      0.71      0.70       479



Metric      : Meaning

Precision   : Out of all predictions for this class, how many were correct?

Recall      : Out of all actual instances of this class, how many did we correctly identify?

F1-Score    : Harmonic mean of Precision and Recall — balances false positives and false negatives.

Support     : Number of actual test samples in each class. Shows class distribution.

# --------------------------------------------------------
## 8) Save Results

In [17]:
# Create necessary folders
import os
os.makedirs("../../Artifacts/models", exist_ok=True)
os.makedirs("../../Artifacts/plots", exist_ok=True)
os.makedirs("../../Artifacts/predictions", exist_ok=True)

In [18]:
# Save TFLite model
converter = tf.lite.TFLiteConverter.from_keras_model(best_model)
tflite_model = converter.convert()
with open("../../Artifacts/models/DL_model.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\edcul\AppData\Local\Temp\tmpznmsftl4\assets


INFO:tensorflow:Assets written to: C:\Users\edcul\AppData\Local\Temp\tmpznmsftl4\assets


Saved artifact at 'C:\Users\edcul\AppData\Local\Temp\tmpznmsftl4'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 14), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  1557255448976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1557254215184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1557281313744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1557281315664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1557281316624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1557281318352: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [19]:
# Save training stats (optional: you can log during training and save)
# Here we'll save mean and std of X_train
train_mean = pd.DataFrame(X_train.mean()).T
train_std = pd.DataFrame(X_train.std()).T

train_mean.to_csv("../../Artifacts/predictions/DL_train_mean.csv", index=False)
train_std.to_csv("../../Artifacts/predictions/DL_train_std.csv", index=False)

In [20]:
# Save confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.savefig("../../Artifacts/plots/DL_confusion_matrix.png")
plt.close()

In [21]:
# Save training accuracy per epoch (retrain with callback to capture it)
history = best_model.fit(X_train, y_train, epochs=20, validation_split=0.2)
plt.figure()
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.tight_layout()
plt.savefig("../../Artifacts/plots/DL_accuracy_per_epoch.png")
plt.close()

Epoch 1/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8522 - loss: 0.4141 - val_accuracy: 0.7102 - val_loss: 0.8767
Epoch 2/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8717 - loss: 0.4236 - val_accuracy: 0.7128 - val_loss: 0.8870
Epoch 3/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8643 - loss: 0.3938 - val_accuracy: 0.6997 - val_loss: 0.8894
Epoch 4/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8699 - loss: 0.3926 - val_accuracy: 0.7154 - val_loss: 0.8994
Epoch 5/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8523 - loss: 0.3983 - val_accuracy: 0.7232 - val_loss: 0.9011
Epoch 6/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8782 - loss: 0.3660 - val_accuracy: 0.7232 - val_loss: 0.9394
Epoch 7/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8533 - loss: 0.3882 - val_accuracy: 0.7050 - val_loss: 0.9186
Epoch 8/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8671 - loss: 0.3779 - val_accuracy: 0.7128 - val_loss: